# Time Series Decomposition

The goal of this material is to introduce basic time series decomposition methods that can be applied to separate the various sub-patterns that are typically combined within time series data.  Many techniques can be used for this purpose and, as an introduction to this general topic, the computations contained herein are aimed, first, at communicating clearly what time series decompositon is and a general approach.

We have seen this image in class previously which nicely visualizes what we will be doing in the two examples included in this Jupyter notebook.

![Cleveland Time Series Decomposition](ClevelandTimeSeriesDecomposition.jpg)

## Demand Pattern Decomposition

This first example is obviously fictious data because its patterns appear to be significantly more consistent across time than  real-world data.  Nonetheless, it is a good first exercise, particualrly, because the easily observed patterns make this first exercise easier than is normally the case.

This data was adapted from the textbook _Supply Chain Management: 3rd Edition_, by Sunil Chopra and Peter Meindl,  Prentice Hall, 2006.

First, let's import all the packages we will need, as well as the magic command for plotting in the Jupyter notebook.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
df = pd.read_csv('TSDataP1.csv')
df

We will need a column for the Quarter index to use with computation, so let's create that now.  We will use a zero-based index. 

Let's look at the data, first, to begin to understand it.

In [ ]:
plt.plot(df.index,df.Product1Demand)
plt.show()

Describe the pattern in this graph?  What "component patterns" do you see within the overall graph?

At this point you should observe an overall (positive) trend  and seasonality.  To perceive the trend, ask what results a linear regression analysis would yield for intercept adn slope.  Perhaps, you will intuit that the trend in this graph is a linear one rather than a nonlinear one.

You could, and should ask yourself what is causing those patterns.  You may know the causes from experience if you are familiar with the context or, alternately, you may need to do research to determine the causes.  If you were a consultant, you might well be in the latter position.

Our goal now is to decompose these data.  Doing so will enable us to forecast demand into the future because the components are easier to detect and express individually, so that we may exptrapolate them into the future.  We need to assume a functional form for how the components contribute to the overall pattern in order to guide the math that we do.  For our pruposes, we will assume this model, which is called an additive model:

- $D$ = Product Demand 
- $q$ = the index of the quarter
- $L$ = The 'level' component of the demand pattern which is a constant value
- $T$ = The (linear) trend of the data.  This is the amount that demand increases, on average, from one quarter to the next.  T is also a constant.
- $S \left(q \right)$ = Seasonality component.  We need to figure out how many quarters there are before the seaonal pattern repeats so that we can determine the seasonality component for each quarter, $q$.
- $\epsilon \left( q \right)$ is the portion of demand that we will not be able to fit with $L$, $T \left(q \right)$ and $S \left(q \right)$.


$D\left(q \right) = L + T \left(q \right) + S \left(q \right) + \epsilon \left( q \right)$ 

We can easily get $L$and $T$ from a linear regression analysis.  We will use a new Python package for this analysis.  Many Python packages can do this analysis well.  

In [ ]:
from scipy import stats

slope, intercept, r_value, p_value, std_err = stats.linregress(df.index,df['Product1Demand'])
print('intercept =', intercept, '    slope =', slope, '     p_value = ',p_value)

Now we need to 'remove' the trend and the level from the original demand pattern to see what pattern remains for use to describe seasonality.  We use the <code>DataFrame.apply()</code> method from pandas for this task.

In [ ]:
def create_regress_col(row, intercept, slope):
    return float(intercept) + row.Quarter * slope
    
df['regress'] = df.apply(create_regress_col,args = (intercept,slope),axis = "columns")
df.style.format({
    'Product1Demand': '{:,.0f}'.format,
    'regress': '{:,.0f}'.format
})

In the computation below, a column is created in the <code>DataFrame</code> whose name is <code>R1</code>.  The <code>R</code> stands for remainder of the original sales trajectory after the regression 'pattern' has been extracted from the original sales pattern.  We still need to find whatever patterns might exist within the remainder.

In [ ]:
df['R1'] = df['Product1Demand'] - df['regress']
df.style.format({
    'Product1Demand': '{:,.0f}'.format,
    'regress': '{:,.0f}'.format,
    'R1': '{:,.0f}'.format
})

Here is a plot of the R1 data column.

In [ ]:
plt.plot(df.index,df.R1)
plt.show()

This looks like a repeated pattern.

How many quarters pass before the pattern repeats?

When the autocorrelation value for a particular lag is large (close to 1) and positive, it indicates a cyclic pattern with the periodicty of that lag.

In [ ]:
for i in range(int(len(df.index)/2)):
    print('autocorrelation, lag =',i,':',df.R1.autocorr(lag = i))

This code plots each sequential series of 4 points, where 4 corresponds with the periodicty of the data.  Note how the patterns have similar shapes, which is why the autocorrelation with this lag was nearly 1.  Let's create a graph that demonstrates this by plotting each successive group of four points.

In [ ]:
dfQtr = pd.DataFrame()
cycleLen = 4
for i in range(int(len(df.index)/cycleLen)):
    newData = pd.DataFrame({i:df['R1'].iloc[i*cycleLen:(i+1)*cycleLen]})
    newData.index = range(0,len(newData))
    dfQtr = pd.concat([dfQtr,newData],axis=1)

fig,ax = plt.subplots()
ax.plot(dfQtr)
plt.show()

If we average the demand for each of the seasonal quarters, so those averages represent all the curves well?

In [ ]:
avg = []
for i in range(len(dfQtr.index)):
    avg.append(dfQtr.iloc[i].mean())

dfQtr = pd.concat([dfQtr,pd.DataFrame({'avg':avg})], axis=1)
print(dfQtr)

fig,ax = plt.subplots()
c = 180
for col in dfQtr.columns.values:
    if col == 'avg':
        ax.plot(dfQtr[col], c = 'r')
    else:
        ax.plot(dfQtr[col], c = 'k')
        
plt.show()

What does the remainder of the demand pattern look like if we use the average seasonality?

In [ ]:
#def calcR2(row):
#    return row[3] - row[4]
    
df['S'] = np.NaN
df['error'] = np.NaN
df['R2'] = np.NaN
df['Forecast'] = np.NaN
S = dfQtr['avg'].tolist()
for i in df.index:
    df.loc[i,'S'] = S[i%cycleLen]

df['R2'] = df['R1'] - df['S']
df['Forecast'] = df['regress'] + df['S']
df['error%'] = 100*abs(df['R2'] / df['Product1Demand'])
df.style.format({
    'Product1Demand': '{:,.0f}'.format,
    'regress': '{:,.0f}'.format,
    'R1': '{:,.0f}'.format,
    'S': '{:,.0f}'.format,
    'R2': '{:,.0f}'.format,
    'Forecast':'{:,.0f}'.format,
    'error': '{:.4f}'.format
})

Let's visualize how our model of demand fits actual product demand

In [ ]:
fig, ax = plt.subplots()
ax.plot(df['Product1Demand'],c='k')
ax.plot(df['Forecast'],c='b')
plt.show()

Here is a plot of the remainder, that is, the error of our model.

In [ ]:
plt.plot(df.index,df.R2)
plt.show()

In [ ]:
plt.plot(df.index,df.R2)
plt.plot(df.index,df.Product1Demand)
plt.show()

Here is a package that does decomposition in one step.

## United States Monthly Home Sales Time Series Decomposition

This data was downloaded form teh Internet (link to be added) and it portrays monthly home sales in the United States on a monthly basis where the data have _not been seasonally adjusted_ as is the case with many data sources.

We will use a similar, but more general model for the Home Sales data.  Specifically, we will use this model:

$D\left(q \right) = T\left( q \right) + C\left( q \right) + \epsilon \left( q \right)$

where

- $D\left( q \right)$ = Product Demand in Quarter $q$
- $q$ = the index of the quarter
- $L$ = The 'level' component of the demand pattern which is a constant value
- $T\left( q \right)$ = $T$ represents the trend of the data which, in this case, could be nonlinear.  $T\left( q \right)$ is the trend value for Quarter $q$.
- $C\left( q \right)$ = Cyclicality demand component, which plays an analogous role to $S\left( q \right)$, except that the pattern repeats after some number of periods which might not be four quarters.
- $\epsilon \left( q \right)$ is the portion of demand that we will not be able to fit with $L$, $T \left(q \right)$ and $S \left(q \right)$.

Often, the ultimate goal of decomposing data like these is to forecast demand into the future because the components are easier to detect and express individually: Those components can be exptrapolated into the future and combined to create a forecast.  






In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

The next cell of code computes the moving average of each point for a data window centered on each point.  The window size is a variable that can be easily changed, and the average squared error is computed in order to help evaluate which window size is appropriate for the moving average.

In [ ]:
def sqErr(row):
    return (row[1] - row[2])**2
    
dates = ['2007-08','2007-09','2007-10','2007-11','2007-12','2008-01','2008-02','2008-03','2008-04','2008-05','2008-06','2008-07','2008-08','2008-09','2008-10','2008-11','2008-12','2009-01','2009-02','2009-03','2009-04','2009-05','2009-06','2009-07','2009-08','2009-09','2009-10','2009-11','2009-12','2010-01','2010-02','2010-03','2010-04','2010-05','2010-06','2010-07','2010-08','2010-09','2010-10','2010-11','2010-12','2011-01','2011-02','2011-03','2011-04','2011-05','2011-06','2011-07','2011-08','2011-09','2011-10','2011-11','2011-12','2012-01','2012-02','2012-03','2012-04','2012-05','2012-06','2012-07','2012-08','2012-09','2012-10','2012-11','2012-12','2013-01','2013-02','2013-03','2013-04','2013-05','2013-06','2013-07','2013-08','2013-09','2013-10','2013-11','2013-12','2014-01','2014-02','2014-03','2014-04','2014-05','2014-06','2014-07','2014-08','2014-09','2014-10','2014-11','2014-12','2015-01','2015-02','2015-03','2015-04','2015-05','2015-06','2015-07','2015-08','2015-09','2015-10','2015-11','2015-12','2016-01','2016-02','2016-03','2016-04','2016-05','2016-06','2016-07','2016-08','2016-09','2016-10','2016-11','2016-12','2017-01','2017-02','2017-03','2017-04','2017-05','2017-06','2017-07','2017-08','2017-09','2017-10','2017-11','2017-12','2018-01','2018-02','2018-03','2018-04','2018-05','2018-06','2018-07','2018-08']
homeSales = [878,674,807,509,1641,701,1048,886,517,661,594,990,597,946,406,358,557,348,509,782,536,638,686,896,520,734,673,760,740,471,344,527,556,565,551,818,552,622,482,415,494,617,704,516,491,466,558,620,523,522,437,403,659,663,422,812,508,624,701,529,629,655,487,516,560,420,403,642,863,652,655,574,559,493,489,369,460,361,357,455,539,610,530,566,521,455,483,387,572,402,386,869,538,612,655,598,462,506,451,418,512,383,336,452,517,572,577,521,549,560,541,574,584,398,361,542,525,624,659,503,548,457,439,431,375,341,422,578,435,502,530,509,220]

dfHS = pd.DataFrame({'YearMonth':dates, 'homeSales':homeSales})
dfHS['MovAvg'] = np.NaN
dfHS['sqErr'] = np.NaN
# Chaging the DataFrame index to DatetimeIndex data type is required for using one of the functions below
dfHS.index = pd.DatetimeIndex(pd.date_range(pd.Timestamp(year=2007, month=8, day=31), periods = len(dfHS['homeSales'])))
#print(len(data),'\n',data)

window = 40
window = window - window % 2
for i in range(int(window/2),dfHS.shape[0]-int(window/2)):
    dfHS.loc[dfHS.index[i],'MovAvg'] = (0.5*dfHS.iloc[i - int(window/2)]['homeSales'] + dfHS.iloc[i - int(window/2)+1:i + int(window/2)]['homeSales'].sum() + 0.5*dfHS.iloc[i + int(window/2)]['homeSales'])/float(window)
dfHS['sqErr'] = dfHS.apply(sqErr,axis='columns')
# The moving average cannot be applied to all rows and we need to delete those rows because we cannot use them in the analysis
dfHS.dropna(how='any',inplace=True)

fig,ax = plt.subplots()
ax.plot(dfHS['MovAvg'],label='Moving Avg.')
ax.plot(dfHS['homeSales'],label='Home Sales')
ax.set_xlabel('Year')
ax.set_ylabel('Units of Demand')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
print('Average Squared Error per Month: ',sum(dfHS['sqErr'])/len(dfHS))
print(dfHS)
plt.show()

The residual Home Sales yet to be explained, $R1$, is computed by subtracting the moving average from the demand time series.  Also, these are included in this code cell:
- Computing $R1$ as a percentage of demand ($R1Error$).
- The dfHS.style.format command demonstrates how to display pandas DataFrame data in whicever readble format you preer.

In [ ]:
dfHS['R1'] = dfHS['homeSales'] - dfHS['MovAvg']
dfHS['R1Error'] = abs((dfHS['homeSales'] - dfHS['R1'])/dfHS['homeSales'])
dfHS.style.format({
    'MovAvg': '{:.1f}'.format,
    'sqErr': '{:,.1f}'.format,
    'R1': '{:,.1f}'.format,
    'R1Error': '{:,.3f}'.format
})

The cell below helps us visualize the remaining pattern to be decomposed, $R1$, and it also computes the average resdiual demadn pattern.

In [ ]:
fig,ax = plt.subplots()
ax.plot(dfHS['R1'])
ax.set_xlabel('Year')
ax.set_ylabel('Units of Demand')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
print('Average Residual: ', sum(dfHS['R1'])/len(dfHS))
plt.show()

Just as seasonal demand had a higher autocorrelation when the data were offet by four periods, we need to use autocorrelation analysis to detect whether any cyclical patterns exist and how many periods before they are repeated.

In [ ]:
maxCorr = 0.0
period = np.NaN
for i in range(1,37):
    corr = dfHS['R1'].autocorr(lag=i)
    print('Correlation, lag ',i,'   ',corr)
    if corr > maxCorr:
        maxCorr = corr
        period = i
print('period = ',period,'     Maximum Correlation = ',maxCorr)

The code cell below breaks the time series into three components corresonding with each of the three cycles in the data.  Note that the third cycle is partial.

In [ ]:
cycleLen = period
numCycles = int(len(dfHS)/cycleLen + 0.5)
cycles = [dfHS.iloc[range(i*period,min((i+1)*period,len(dfHS)))]['R1'] for i in range(numCycles)]
fig,ax = plt.subplots()
for i in range(len(cycles)):
    ax.plot(cycles[i].values,label='Cycle '+str(i))
ax.set_xlabel('Month')
ax.set_ylabel('Units of Demand')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.legend()
plt.show()

This code cell:
- Computes the average home sales for each of the periods in the cycle.  that is, it computes $C\left( q \right)$
- Inserts the appropriate $C\left( q \right)$ value into the $C$ column in the DataFrame for each Quarter $q$.
- Finally, a plot of the cyclicality component $C\left( q \right)$ is plotted with the $R1$ column to see how well the cyclicality component and it match.

In [ ]:
cycleLen = period   # see prior cell for computation of cyclicality period
avg = []            # a list to store the average demand for each period of the cycle
numCycles = int(len(dfHS)/cycleLen + 0.5)
for j in range(cycleLen):
    if j + (numCycles-1) * cycleLen < len(dfHS):
        d = dfHS.iloc[range(j,j + (numCycles-1) * cycleLen+1,cycleLen)]['R1']
        #print(j,d)
        avg.append(sum(d)/len(d))
    else:
        d = dfHS.iloc[range(j,j + (numCycles-2) * cycleLen+1,cycleLen)]['R1']
        #print(j,d)
        avg.append(sum(d)/len(d))
dfHS['C'] = np.NaN
for i in range(len(dfHS)):
    dfHS.loc[dfHS.index[i], 'C'] = avg[i % cycleLen]

fig,ax = plt.subplots()
ax.plot(dfHS['C'],label='Cyclic Pattern')
ax.plot(dfHS['R1'],label='Remainder After Trend')
ax.set_xlabel('Year')
ax.set_ylabel('Units of Demand')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
fig.legend()
plt.show()

Another view of the cyclical component we've approximated versus each of the three cycles.

In [ ]:
fig,ax = plt.subplots()
for i in range(len(cycles)):
    ax.plot(cycles[i].values,label='Cycle '+str(i),c='k')
ax.plot(avg,label='Average Cycle',c='r')
ax.set_xlabel('Month')
ax.set_ylabel('Units of Demand')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.legend()
plt.show()

The code cell below does these operations:

- Computes the remaining residual home sales to be explained, $R2$, after subtracting the cyclical component, $C \left( q \right)$. 
- Computes the remainder after cyclicality, $R2$ as a percentage of the demand time series.
- Computes the mathematical model 'fit', composed of the trend and cyclical components: $T\left( q \right) + C\left( q \right)$.
- Plots the fit of the model $T\left( q \right) + C\left( q \right)$ with the original data, $D \left( q \right)$.
- Computes the average absolute error of $R2Error$ of the original demand time series.

In [ ]:
dfHS['R2'] = dfHS['R1'] - dfHS['C']
dfHS['R2Error'] = abs(dfHS['R2']/dfHS['homeSales'])
dfHS['fit'] = dfHS['MovAvg'] + dfHS['C']
print(dfHS)
fig,ax = plt.subplots()
ax.plot(dfHS['homeSales'],label='Home Sales')
ax.plot(dfHS['fit'], label = 'Fit')
ax.set_xlabel('Year')
ax.set_ylabel('Units of Demand')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
fig.legend()
print('Average Error: ', sum(dfHS['R2Error'])/len(dfHS))
plt.show()

Here is a plot of the residual $R2$ for visualization purposes to observe any remaining patterns that we might want to capture, and also an autocorrelation analysis of the residual.

In [ ]:
fig,ax = plt.subplots()
ax.plot(dfHS['R2'],label='Remainder after Trend and Cyclical Components')
ax.set_xlabel('Year')
ax.set_ylabel('Units of Demand')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
fig.legend()
maxCorr = 0.0
period = np.NaN
for i in range(1,37):
    corr = dfHS['R2'].autocorr(lag=i)
    print('Correlation, lag ',i,'   ',corr)
    if corr > maxCorr:
        maxCorr = corr
        period = i
print('period = ',period,'     Maximum Correlation = ',maxCorr)
plt.show()

A final graph to show the model versus the original data and, as well, the remander $R2$ to judge it relative to the original demand we were trying to fit.

In [ ]:
fig,ax = plt.subplots()
ax.plot(dfHS['homeSales'],label='Home Sales')
ax.plot(dfHS['fit'],label='Fit')
ax.plot(dfHS['R2'],label='Residual')
ax.set_xlabel('Year')
ax.set_ylabel('Units of Demand')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
fig.legend()
plt.show()

Here is a package that performs time series decomposition in one statement, although you need to do some digging into the results for figure out how the model was constructed.  This is the computation that requires the pandas DataFrame index to be of the DatetimeIndex data type.

In [ ]:
import statsmodels.api as sm

res = sm.tsa.seasonal_decompose(dfHS['homeSales'],period=36)
res.plot()

## Another Approach to Monthly Home Sales Time Series Decomposition

The prior approach to decomposing monthly home sales works well, but it is not amenable to a forecast of future sales for the reasons we will discuss in class.  The approach used in this section differs only by how the trend cmoponent is computed, which is a moving average of some number of periods prior to the perio od sales we are trying to forecast.  That is, it uses only data from the past relative to a point to compute the forecast for that point.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def sqErr(row):
    return (row[1] - row[2])**2
    
dates = ['2007-08','2007-09','2007-10','2007-11','2007-12','2008-01','2008-02','2008-03','2008-04','2008-05','2008-06','2008-07','2008-08','2008-09','2008-10','2008-11','2008-12','2009-01','2009-02','2009-03','2009-04','2009-05','2009-06','2009-07','2009-08','2009-09','2009-10','2009-11','2009-12','2010-01','2010-02','2010-03','2010-04','2010-05','2010-06','2010-07','2010-08','2010-09','2010-10','2010-11','2010-12','2011-01','2011-02','2011-03','2011-04','2011-05','2011-06','2011-07','2011-08','2011-09','2011-10','2011-11','2011-12','2012-01','2012-02','2012-03','2012-04','2012-05','2012-06','2012-07','2012-08','2012-09','2012-10','2012-11','2012-12','2013-01','2013-02','2013-03','2013-04','2013-05','2013-06','2013-07','2013-08','2013-09','2013-10','2013-11','2013-12','2014-01','2014-02','2014-03','2014-04','2014-05','2014-06','2014-07','2014-08','2014-09','2014-10','2014-11','2014-12','2015-01','2015-02','2015-03','2015-04','2015-05','2015-06','2015-07','2015-08','2015-09','2015-10','2015-11','2015-12','2016-01','2016-02','2016-03','2016-04','2016-05','2016-06','2016-07','2016-08','2016-09','2016-10','2016-11','2016-12','2017-01','2017-02','2017-03','2017-04','2017-05','2017-06','2017-07','2017-08','2017-09','2017-10','2017-11','2017-12','2018-01','2018-02','2018-03','2018-04','2018-05','2018-06','2018-07','2018-08']
homeSales = [878,674,807,509,1641,701,1048,886,517,661,594,990,597,946,406,358,557,348,509,782,536,638,686,896,520,734,673,760,740,471,344,527,556,565,551,818,552,622,482,415,494,617,704,516,491,466,558,620,523,522,437,403,659,663,422,812,508,624,701,529,629,655,487,516,560,420,403,642,863,652,655,574,559,493,489,369,460,361,357,455,539,610,530,566,521,455,483,387,572,402,386,869,538,612,655,598,462,506,451,418,512,383,336,452,517,572,577,521,549,560,541,574,584,398,361,542,525,624,659,503,548,457,439,431,375,341,422,578,435,502,530,509,220]

dfHSa = pd.DataFrame({'YearMonth':dates, 'homeSales':homeSales})
dfHSa['MovAvg'] = np.NaN
dfHSa['sqErr'] = np.NaN
dfHSa.index = pd.DatetimeIndex(pd.date_range(pd.Timestamp(year=2007, month=8, day=31), periods = len(dfHSa['homeSales'])))
#print(len(data),'\n',data)

window = 30
for i in range(window+1,len(dfHSa)):
    dfHSa.loc[dfHSa.index[i],'MovAvg'] = sum(dfHSa.iloc[range(i-window-1,i)]['homeSales'])/float(window)
dfHSa['sqErr'] = dfHSa.apply(sqErr,axis='columns')
#print(data.head())
dfHSa.dropna(how='any',inplace=True)

fig,ax = plt.subplots()
ax.plot(dfHSa['MovAvg'], label='Moving Avg.')
ax.plot(dfHSa['homeSales'], label='Home Sales')
ax.set_xlabel('Year')
ax.set_ylabel('Units of Demand')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
fig.legend()
plt.show()
print('Average Squared Error per Month: ',sum(dfHSa['sqErr'])/len(dfHSa))
print(dfHSa)

In [ ]:
dfHSa['R1'] = dfHSa['homeSales'] - dfHSa['MovAvg']
dfHSa['R1Error'] = abs((dfHSa['homeSales'] - dfHSa['R1'])/dfHSa['homeSales'])
dfHSa.style.format({
    'MovAvg': '{:.1f}'.format,
    'sqErr': '{:,.1f}'.format    
})

In [ ]:
fig,ax = plt.subplots()
ax.plot(dfHSa['R1'],label='Remainder after Trend')
ax.set_xlabel('Year')
ax.set_ylabel('Units of Demand')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
fig.legend()
plt.show()
print('Average Residual: ', sum(dfHSa['R1'])/len(dfHSa))

maxCorr = 0.0
period = np.NaN
for i in range(1,37):
    corr = dfHSa['R1'].autocorr(lag=i)
    print('Correlation, lag ',i,'   ',corr)
    if corr > maxCorr:
        maxCorr = corr
        period = i
print('period = ',period,'     Maximum Correlation = ',maxCorr)

In [ ]:
cycleLen = period   # see prior cell for computation of cyclicality period
avg = []            # a list to store the average demand for each period of the cycle
numCycles = int(len(dfHSa)/cycleLen + 0.5)
for j in range(cycleLen):
    if j + (numCycles-1) * cycleLen < len(dfHSa):
        d = dfHSa.iloc[range(j,j + (numCycles-1) * cycleLen+1,cycleLen)]['R1']
        print(j,d)
        avg.append(sum(d)/len(d))
    else:
        d = dfHSa.iloc[range(j,j + (numCycles-2) * cycleLen+1,cycleLen)]['R1']
        print(j,d)
        avg.append(sum(d)/len(d))
dfHSa['C'] = np.NaN
for i in range(len(dfHSa)):
    dfHSa.loc[dfHSa.index[i], 'C'] = avg[i % cycleLen]

fig,ax = plt.subplots()
ax.plot(dfHSa['C'],label='Cyclic Pattern')
ax.plot(dfHSa['R1'],label='Remainder After Trend')
ax.set_xlabel('Year')
ax.set_ylabel('Units of Demand')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
fig.legend()
plt.show()

In [ ]:
fig,ax = plt.subplots()
for i in range(len(cycles)):
    ax.plot(cycles[i].values,label='Cycle '+str(i),c='k')
ax.plot(avg,label='Average Cycle',c='r')
ax.set_xlabel('Month')
ax.set_ylabel('Units of Demand')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.legend()
plt.show()

In [ ]:
dfHSa['R2'] = dfHSa['R1'] - dfHSa['C']
dfHSa['R2Error'] = abs(dfHSa['R2']/dfHSa['homeSales'])
dfHSa['fit'] = dfHSa['MovAvg'] + dfHSa['C']
print(dfHSa)
fig,ax = plt.subplots()
ax.plot(dfHSa['homeSales'],label='Home Sales')
ax.plot(dfHSa['fit'], label = 'Fit')
ax.set_xlabel('Year')
ax.set_ylabel('Units of Demand')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
fig.legend()
plt.show()
print('Average Error: ', sum(dfHSa['R2Error']/len(dfHSa)))

In [ ]:
fig,ax = plt.subplots()
ax.plot(dfHSa['R2'], label='Remainder after Trend and Cyclic Components')
ax.set_xlabel('Year')
ax.set_ylabel('Units of Demand')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
fig.legend()
plt.show()

maxCorr = 0.0
period = np.NaN
for i in range(1,37):
    corr = dfHSa['R2'].autocorr(lag=i)
    print('Correlation, lag ',i,'   ',corr)
    if corr > maxCorr:
        maxCorr = corr
        period = i
print('period = ',period,'     Maximum Correlation = ',maxCorr)

In [ ]:
fig,ax = plt.subplots()
ax.plot(dfHSa['homeSales'],label='Home Sales')
ax.plot(dfHSa['fit'],label='Fit')
ax.plot(dfHSa['R2'],label='Residual')
ax.set_xlabel('Year')
ax.set_ylabel('Units of Demand')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
fig.legend()
plt.show()